In [2]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from Positive_transform import Positive_transform
from sklearn.preprocessing import StandardScaler
from normalRegressionLayer import NormalRegressionLayer
import pickle
from Test_error_summary import Test_error_summary
import sys
import argparse
import warnings
warnings.filterwarnings("ignore")

parser = argparse.ArgumentParser('nneTrain')
# neural network algorithm/training settings
parser.add_argument('--num_nodes', type=int, help='layer width', default=128)   # 128
parser.add_argument('--batch_size', type=int, help='training sample batch', default=64)    # 64
parser.add_argument('--max_epochs', type=int, help='training epoches', default=200)         # 200
parser.add_argument('--initial_lr', type=int, help='initial learning rate', default=0.01)   # 0.01

# display settings
parser.add_argument('--disp_test_summary', type=bool, help='display test summary', default=True)
parser.add_argument('--display_fig', type=bool, help='display figure', default=True)
parser.add_argument('--disp_iter', type=bool, help='display iteration', default=True)
parser.add_argument('--learn_standard_error', type=bool, help='learn standard error', default=True)

parser.add_argument('-f', type=str, default=None, 
                    help=argparse.SUPPRESS)  # SUPPRESS 让它不出现在帮助文档中

try:
    args = parser.parse_args()
except:
    parser.print_help()
    sys.exit(0)

In [3]:
def set_train_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

class NeuralNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        return self.model(x)


def forward_loss(Y, T): # y_pred, y_true
    #print(Y.shape)
    #print(T.shape)
    k = Y.shape[1] // 2
    n = Y.shape[0]
    U = Y[:, :k]
    S = torch.tensor(Positive_transform(Y[:, k:2*k].detach().numpy()))
    X = T[:, :k]
    squared_err = 2 * torch.log(S) + ((U - X) / S) ** 2
    # sum all elements, then divide by n
    loss = squared_err.sum() / n
    return loss

In [4]:
with open('simu_data_collect/model2/training_set_gen.pkl', 'rb') as f:
#with open('training_set_gen.pkl', 'rb') as f:  # data from "set_up.py"
    data = pickle.load(f)
set_train_seed(101)

In [5]:
# load training/validation/real data
input_train, label_train = data['input_train'], data['label_train']
input_test, label_test = data['input_test'], data['label_test']
input_real = data['input_real']
lb, ub = data['lb'], data['ub']
label_name = data['label_name']

In [9]:
'''
Moment list:
1. (mean_deg, var_deg, clusterglobal)*period,
2. (mean(y_ijt, x_ij,t-1), cov(y_ijt, x_ij,t-1))*period,
x_ij,t-1 = (y_ij,t-1, SumDegree_ij,t-1, NetDistanceij,t-1)
3. zi (feature)
'''

print(input_train.shape, input_test.shape, input_real.shape) # (n_samples, moment_dim), (n_samples, moment_dim), (moment_dim,)

(900, 31) (100, 31) (31,)


In [7]:
input_real

array([ 1.13659896e+00,  2.00005837e+00,  3.34075724e-03,  1.36121067e+00,
        2.56367261e+00,  6.28272251e-03,  1.54201513e+00,  3.07818296e+00,
        7.97219951e-03,  1.71007567e+00,  3.65867586e+00,  8.71137410e-03,
        5.36497314e-04,  9.72694425e-01,  1.32381622e+00,  5.36205289e-04,
       -2.53090945e-04,  7.14624277e-04,  2.61605953e-03, -2.68053329e-02,
        7.18957948e-01,  8.24175523e-03,  9.68878036e-01,  1.33659432e+00,
        8.17282901e-03, -3.85616901e-03,  1.08428928e-02,  4.31468626e-03,
       -3.15861976e-02,  7.34796328e-01,  2.65586587e-02])

In [10]:
R_train, R_test = input_train.shape[0], input_test.shape[0] # num of train/test 'samples'
R = R_train + R_test
M, L = input_train.shape[1], label_train.shape[1]

if args.learn_standard_error:   # sd as labels
    label_train = np.hstack([label_train, np.zeros((R_train, L))])
    label_test = np.hstack([label_test, np.zeros((R_test, L))])
    output_dim = 2 * L
else:
    output_dim = L

scaler = StandardScaler()
input_train, input_test = scaler.fit_transform(input_train), scaler.fit_transform(input_test)
input_real = scaler.fit_transform(input_real.reshape(1, -1))

# Dataset and DataLoader
train_dataset = TensorDataset(torch.tensor(input_train, dtype=torch.float32),
                              torch.tensor(label_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
net = NeuralNet(M, args.num_nodes, output_dim)


device = torch.device("cpu")
net.to(device)
if args.learn_standard_error:
    criterion = forward_loss
else:
    criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=args.initial_lr)

# Optional: learning rate scheduler similar to MATLAB piecewise schedule
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1) # 40, 0.1

for epoch in range(args.max_epochs):
    net.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = net(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

    scheduler.step()

# Validation / Test summary (optional)
    net.eval()
    with torch.no_grad():
        test_preds = net(torch.tensor(input_test, dtype=torch.float32))
        loss_va = criterion(test_preds, torch.tensor(label_test, dtype=torch.float32))

    if args.disp_iter:
        print(f"Epoch {epoch + 1}/{args.max_epochs}, Train Loss: {loss.item():.4f}, Val Loss: {loss_va.item():.4f}")


if args.disp_test_summary:
    net.eval()
    with torch.no_grad():
        test_preds = net(torch.tensor(input_test, dtype=torch.float32))
        Test_error_summary(torch.tensor(input_test, dtype=torch.float32), label_test, label_name, net, figure=args.display_fig, table=1)

# Estimate on original data
net.eval()
with torch.no_grad():
    temp = net(torch.tensor(input_real.squeeze(0), dtype=torch.float32)).numpy()

# 截断 theta
theta = np.clip(temp[:L], lb, ub)

# 正向变换标准误（如果学习）
if args.learn_standard_error:
    se = Positive_transform(temp[L:2 * L])

print(theta)
print(se)

Epoch 1/200, Train Loss: 133.5827, Val Loss: 114.0630
Epoch 2/200, Train Loss: 77.7617, Val Loss: 341.9515
Epoch 3/200, Train Loss: 78.4388, Val Loss: 65.8587
Epoch 4/200, Train Loss: 59.3914, Val Loss: 50.5181
Epoch 5/200, Train Loss: 74.1005, Val Loss: 47.6608
Epoch 6/200, Train Loss: 22.7167, Val Loss: 42.6580
Epoch 7/200, Train Loss: 7.0540, Val Loss: 40.3200
Epoch 8/200, Train Loss: 57.6607, Val Loss: 38.3525
Epoch 9/200, Train Loss: 23.6220, Val Loss: 39.3016
Epoch 10/200, Train Loss: 33.8099, Val Loss: 40.3671
Epoch 11/200, Train Loss: 0.2291, Val Loss: 38.9996
Epoch 12/200, Train Loss: 25.6668, Val Loss: 36.6207
Epoch 13/200, Train Loss: 36.5096, Val Loss: 42.0711
Epoch 14/200, Train Loss: 37.9546, Val Loss: 36.9787
Epoch 15/200, Train Loss: 7.7854, Val Loss: 43.7293
Epoch 16/200, Train Loss: 20.2997, Val Loss: 46.7072
Epoch 17/200, Train Loss: 30.7493, Val Loss: 49.1016
Epoch 18/200, Train Loss: 40.8090, Val Loss: 43.2020
Epoch 19/200, Train Loss: 52.7868, Val Loss: 38.8965
Ep